# EDA - Finger Vein Project (v5, Enhanced-Combined)

ข้อมูลที่ใช้ใน EDA นี้มาจากโฟลเดอร์:
```
D:/Finger-Vein-Scan/data/Enhanced-Combined/
```

ไฟล์ CSV ที่ใช้อ่านข้อมูลคือ:
```
split_combined.csv
```

### เป้าหมาย:
- ตรวจสอบจำนวนภาพและจำนวนคลาส (ID)
- วิเคราะห์การกระจายของข้อมูลแต่ละชุด (train, val, test)
- แสดงตัวอย่างภาพจากหลายคลาส
- วิเคราะห์ความหลากหลายของ vein pattern ภายในและระหว่างคลาส


In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import os
from PIL import Image
import random

# โหลดข้อมูลจาก split_combined.csv
df = pd.read_csv("split_combined.csv")

# ตรวจสอบข้อมูลเบื้องต้น
df.head()


In [ ]:
# จำนวนภาพทั้งหมด
print(f"Total images: {len(df)}")

# จำนวน ID ทั้งหมด
print(f"Total unique IDs: {df['id'].nunique()}")

# จำนวนภาพต่อ split
print("\nImage count per split:")
print(df['split'].value_counts())

# วาดกราฟจำนวนภาพต่อ split
plt.figure(figsize=(5,3))
sns.countplot(data=df, x='split', order=['train', 'val', 'test'])
plt.title("Image Count per Split")
plt.show()


In [ ]:
# จำนวนภาพต่อ class (label)
class_counts = df['id'].value_counts().sort_index()

# วาด histogram
plt.figure(figsize=(10,4))
sns.histplot(class_counts, bins=50, kde=False)
plt.xlabel("Class ID")
plt.ylabel("Number of Images")
plt.title("Number of Images per Class")
plt.show()


In [ ]:
def show_images(df, num_classes=5, images_per_class=3):
    sample_classes = random.sample(df['id'].unique().tolist(), num_classes)
    fig, axes = plt.subplots(num_classes, images_per_class, figsize=(images_per_class*2, num_classes*2))
    for i, cls in enumerate(sample_classes):
        samples = df[df['id'] == cls].sample(images_per_class)
        for j, path in enumerate(samples['filepath']):
            img = Image.open(path).convert("L")
            axes[i][j].imshow(img, cmap='gray')
            axes[i][j].axis('off')
            if j == 0:
                axes[i][j].set_title(f"Class {cls}")
    plt.tight_layout()
    plt.show()

show_images(df)


In [ ]:
def show_variation(df, class_id=None):
    if class_id is None:
        class_id = random.choice(df['id'].unique())
    same_class_imgs = df[df['id'] == class_id].sample(3)
    diff_class_imgs = df[df['id'] != class_id].sample(3)

    fig, axes = plt.subplots(2, 3, figsize=(9, 6))
    for i, path in enumerate(same_class_imgs['filepath']):
        img = Image.open(path).convert("L")
        axes[0][i].imshow(img, cmap='gray')
        axes[0][i].set_title(f"Same Class {class_id}")
        axes[0][i].axis('off')

    for i, path in enumerate(diff_class_imgs['filepath']):
        img = Image.open(path).convert("L")
        axes[1][i].imshow(img, cmap='gray')
        axes[1][i].set_title(f"Different Class")
        axes[1][i].axis('off')

    plt.tight_layout()
    plt.show()

show_variation(df)


## 🔍 สรุปผลจาก EDA

- ข้อมูลจากชุด Enhanced-Combined มีการแบ่งเป็น train / val / test ชัดเจน
- จำนวนภาพต่อ ID อยู่ในช่วงคงที่ (เช่น 100 ภาพต่อ ID)
- Vein pattern ภายในคลาสมีลักษณะคล้ายกัน
- คลาสต่าง ๆ มี variation ชัดเจน เหมาะสำหรับการเรียนรู้ด้วยโมเดล deep learning

พร้อมสำหรับนำไปใช้ฝึกโมเดล CNN, ViT, EfficientNet ต่อไป
